<a href="https://colab.research.google.com/github/AleksandreBakhtadze/ML-abakh22-facial-expression-recognition/blob/main/facial_expression_train2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q kaggle
!pip install -q wandb
!pip install torchmetrics

In [ ]:
import wandb
wandb.login()
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c challenges-in-representation-learning-facial-expression-recognition-challenge
!unzip -q challenges-in-representation-learning-facial-expression-recognition-challenge.zip

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: abakh22 (abakh22-free-university-of-tbilisi-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


 86% 246M/285M [00:01<00:00, 132MB/s] 
100% 285M/285M [00:03<00:00, 77.6MB/s]


In [ ]:
# Import libraries
import os
import torch
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
import torch.nn as nn
import torch.optim as optim
from torchmetrics.classification import MulticlassConfusionMatrix, MulticlassPrecision, MulticlassRecall, MulticlassF1Score
import matplotlib.pyplot as plt
import seaborn as sns
from torchsummary import summary
import numpy as np
from PIL import Image

In [13]:
import pandas as pd

# Load the CSV
df = pd.read_csv("train.csv")

# Example row
print(df.head())

# Convert the pixel values to image tensors
def process_row(row):
    pixels = np.array([int(p) for p in row['pixels'].split()], dtype=np.uint8).reshape(48, 48)
    img = Image.fromarray(pixels)
    return img, int(row['emotion'])

images, labels = zip(*[process_row(row) for _, row in df.iterrows()])

   emotion                                             pixels
0        0  70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...
1        0  151 150 147 155 148 133 111 140 170 174 182 15...
2        2  231 212 156 164 174 138 161 173 182 200 106 38...
3        4  24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...
4        6  4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...


In [14]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

class FERDataset(torch.utils.data.Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        img = self.images[idx]
        if self.transform:
            img = self.transform(img)
        return img, self.labels[idx]

# Split into train and val
from sklearn.model_selection import train_test_split
train_imgs, val_imgs, train_labels, val_labels = train_test_split(images, labels, test_size=0.1, stratify=labels)

train_dataset = FERDataset(train_imgs, train_labels, transform)
val_dataset = FERDataset(val_imgs, val_labels, transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64)


In [15]:
class ImprovedCNN(nn.Module):
    def __init__(self):
        super(ImprovedCNN, self).__init__()
        self.conv_block1 = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.conv_block2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.conv_block3 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Sequential(
            nn.Linear(256 * 6 * 6, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 7)  # Assuming 7 emotion classes
        )

    def forward(self, x):
        x = self.conv_block1(x)
        x = self.conv_block2(x)
        x = self.conv_block3(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x


model = ImprovedCNN().to('cuda' if torch.cuda.is_available() else 'gpu')
summary(model, (1, 48, 48))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 48, 48]             640
       BatchNorm2d-2           [-1, 64, 48, 48]             128
              ReLU-3           [-1, 64, 48, 48]               0
         MaxPool2d-4           [-1, 64, 24, 24]               0
            Conv2d-5          [-1, 128, 24, 24]          73,856
       BatchNorm2d-6          [-1, 128, 24, 24]             256
              ReLU-7          [-1, 128, 24, 24]               0
         MaxPool2d-8          [-1, 128, 12, 12]               0
            Conv2d-9          [-1, 256, 12, 12]         295,168
      BatchNorm2d-10          [-1, 256, 12, 12]             512
             ReLU-11          [-1, 256, 12, 12]               0
        MaxPool2d-12            [-1, 256, 6, 6]               0
           Linear-13                  [-1, 512]       4,719,104
             ReLU-14                  [

In [16]:
# Sample 20 training examples
small_dataset, _ = torch.utils.data.random_split(train_dataset, [20, len(train_dataset) - 20])
small_loader = torch.utils.data.DataLoader(small_dataset, batch_size=4, shuffle=True)

# Re-initialize the model
model = ImprovedCNN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

print("Training on a tiny dataset to check overfitting...")
for epoch in range(20):
    model.train()
    total_loss, correct = 0, 0
    for imgs, labels in small_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        outputs = model(imgs)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        correct += (outputs.argmax(1) == labels).sum().item()

    acc = correct / len(small_loader.dataset)
    print(f"Epoch {epoch+1} - Loss: {total_loss:.4f}, Acc: {acc:.4f}")
    if acc == 1.0:
        print("✅ Model successfully overfit tiny dataset")
        break


Training on a tiny dataset to check overfitting...
Epoch 1 - Loss: 35.5255, Acc: 0.4000
Epoch 2 - Loss: 30.5489, Acc: 0.3500
Epoch 3 - Loss: 16.8105, Acc: 0.5000
Epoch 4 - Loss: 9.2955, Acc: 0.5500
Epoch 5 - Loss: 13.0531, Acc: 0.4500
Epoch 6 - Loss: 14.3013, Acc: 0.6500
Epoch 7 - Loss: 13.5415, Acc: 0.5500
Epoch 8 - Loss: 5.1781, Acc: 0.7000
Epoch 9 - Loss: 9.5160, Acc: 0.7000
Epoch 10 - Loss: 1.6484, Acc: 0.9000
Epoch 11 - Loss: 4.1305, Acc: 0.8500
Epoch 12 - Loss: 2.6446, Acc: 0.8500
Epoch 13 - Loss: 3.1666, Acc: 0.9000
Epoch 14 - Loss: 0.0803, Acc: 1.0000
✅ Model successfully overfit tiny dataset


In [17]:
import wandb
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Start Wandb
wandb.init(project="FER-CNN", name="simple_cnn_run")

# Loss & optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(10):
    model.train()
    train_loss, correct = 0, 0
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        outputs = model(imgs)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        correct += (outputs.argmax(1) == labels).sum().item()

    train_acc = correct / len(train_loader.dataset)

    # Validation phase
    model.eval()
    val_loss, val_correct = 0, 0
    all_preds, all_labels = [], []
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            preds = outputs.argmax(1)
            val_correct += (preds == labels).sum().item()
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    val_acc = val_correct / len(val_loader.dataset)

    wandb.log({
        "epoch": epoch + 1,
        "train_loss": train_loss,
        "train_acc": train_acc,
        "val_loss": val_loss,
        "val_acc": val_acc
    })

    print(f"Epoch {epoch+1} - Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}")

# Confusion matrix and classification report at the end
cm = confusion_matrix(all_labels, all_preds)
fig, ax = plt.subplots(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax)
ax.set_xlabel("Predicted")
ax.set_ylabel("True")
ax.set_title("Confusion Matrix")
wandb.log({"confusion_matrix": wandb.Image(fig)})
plt.close(fig)

# Classification report
report = classification_report(all_labels, all_preds, digits=4)
print(report)
wandb.run.summary["classification_report"] = report

# Sample predictions
model.eval()
for i in range(5):
    img, label = val_dataset[i]
    with torch.no_grad():
        pred = model(img.unsqueeze(0).to(device)).argmax(1).item()
    img_np = img.squeeze().numpy()
    fig, ax = plt.subplots()
    ax.imshow(img_np, cmap="gray")
    ax.set_title(f"Predicted: {pred}, True: {label}")
    ax.axis('off')
    wandb.log({f"Example_{i}": wandb.Image(fig)})
    plt.close(fig)

wandb.finish()


Epoch 1 - Train Acc: 0.2689, Val Acc: 0.3542
Epoch 2 - Train Acc: 0.3518, Val Acc: 0.4016
Epoch 3 - Train Acc: 0.3982, Val Acc: 0.4713
Epoch 4 - Train Acc: 0.4185, Val Acc: 0.4974
Epoch 5 - Train Acc: 0.4423, Val Acc: 0.4929
Epoch 6 - Train Acc: 0.4611, Val Acc: 0.5023
Epoch 7 - Train Acc: 0.4761, Val Acc: 0.5068
Epoch 8 - Train Acc: 0.4858, Val Acc: 0.5082
Epoch 9 - Train Acc: 0.4983, Val Acc: 0.5420
Epoch 10 - Train Acc: 0.5045, Val Acc: 0.5482
              precision    recall  f1-score   support

           0     0.4651    0.4010    0.4307       399
           1     0.0000    0.0000    0.0000        44
           2     0.3846    0.1220    0.1852       410
           3     0.7736    0.8047    0.7889       722
           4     0.3758    0.5735    0.4541       483
           5     0.7464    0.6593    0.7002       317
           6     0.4722    0.5988    0.5280       496

    accuracy                         0.5482      2871
   macro avg     0.4597    0.4513    0.4410      2871
weighte

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


epoch,▁▂▃▃▄▅▆▆▇█
train_acc,▁▃▅▅▆▇▇▇██
train_loss,█▅▄▃▃▂▂▂▁▁
val_acc,▁▃▅▆▆▆▇▇██
val_loss,█▆▃▃▃▂▂▂▁▁
classification_report,precis...
epoch,10
train_acc,0.50453
train_loss,503.31711
val_acc,0.54824
val_loss,54.97838


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
